# Proyecto Final

## Programación

## Profesor: Ing. Andrés Mena A.

### **Nombre del estudiante: Stephanny Garita Vega**

### **Objetivo principal:**

“Analizar el comportamiento inicial de los prospectos ingresados durante la campaña del IIIQ 2025, identificando tendencias en fecha de ingreso, carreras de interés, origen de captación, base académica y estado del primer contacto, con el fin de comprender mejor el perfil de los leads y orientar estrategias de marketing y seguimiento.”

Con esto podemos analizar:

✔ Qué carreras tienen más interés
✔ Qué canal trae más prospectos
✔ Cuántos prospectos llegan por día / semana
✔ Distribución por nivel académico
✔ Prospectos trabajados vs no trabajados
✔ Análisis por base (IIIQ, etc.)
✔ Tendencias de ingreso

In [17]:
pip install pandas openpyxl numpy matplotlib

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [16]:
import pandas as pd 

df = pd.read_excel("prospectos.xlsx")
display(df.head())

,Fecha_ingreso,Nombre,Estatus,Contador,Origen,Etapa,Carrera_interés,Fecha_modificado,Base,Grado_académico
0,2025-11-27 00:00:00,Wilfredo Córdova,WhatsApp (NO_CONTACTO 1),1,Formulario Web Google,Trabajado,Bachillerato Derecho,2025-11-27 10:04:27,Nuevo IIIQ,Bachillerato
1,2025-11-27 08:24:35,Steff,WhatsApp (NO_CONTACTO 1),1,Formulario Web Google,Trabajado,Técnico Banca y Finanzas,2025-11-27 10:02:00,Nuevo IIIQ,Técnicos
2,2025-11-27 06:56:57,Andrey Varela,WhatsApp (NO_CONTACTO 1),1,Formulario Web Google,Trabajado,Cisco (CCNA),2025-11-27 10:03:35,Nuevo IIIQ,Técnicos
3,2025-11-26 23:32:19,Oscar josue Lopez dominguez,WhatsApp (NO_CONTACTO 1),1,Formulario Web Google,Trabajado,Cisco (CCNA),2025-11-27 10:05:23,Nuevo IIIQ,Técnicos
4,2025-11-26 22:19:47,Nicolás felipe Calle pinzon,WhatsApp (NO_CONTACTO 1),1,Formulario Web Google,Trabajado,Técnico Mercadeo y Ventas,2025-11-27 10:06:34,Nuevo IIIQ,Técnicos
